In [ ]:
import jax
import jax.numpy as jnp
import optax
import flax.linen as nn
from flax.training import train_state
from typing import Tuple, List
from optax import ScalarOrSchedule
from syuron import dataset
from syuron import mlp


class MLP(nn.Module):
    hidden_sizes: List[int]
    output_size: int

    @nn.compact
    def __call__(self, x):
        for h in self.hidden_sizes:
            x = nn.Dense(features=h)(x)
            x = nn.relu(x)
        x = nn.Dense(features=self.output_size)(x)
        return x


def use_state(learning_rate: ScalarOrSchedule, input_size: int, hidden_sizes: List[int], output_size: int) -> mlp.ModelState:
    """
    MLPモデルを生成し、初期パラメータをダミー入力から初期化。
    Adamオプティマイザを用いてTrainState（ModelState）を返す。
    """
    model = MLP(hidden_sizes=hidden_sizes, output_size=output_size)
    rng = jax.random.PRNGKey(0)
    dummy_input = jnp.ones([1, input_size])
    params = model.init(rng, dummy_input)
    tx = optax.adam(learning_rate)
    state = train_state.TrainState.create(
        apply_fn=model.apply, params=params, tx=tx)
    return state


def loss_fn(params: mlp.ModelParams, batch: dataset.Batch, apply_fn: mlp.ApplyFn) -> mlp.Loss:
    """
    クロスエントロピー損失を計算する。
    モデルの出力(logits)に対してlog_softmaxを適用し、one-hotラベルとのクロスエントロピーの平均を返す。
    """
    logits = apply_fn(params, batch.inputs)
    log_probs = jax.nn.log_softmax(logits)
    loss = -jnp.mean(jnp.sum(batch.outputs * log_probs, axis=-1))
    return loss  # type: ignore


def train_step(state: mlp.ModelState, batch: dataset.Batch) -> Tuple[mlp.ModelState, mlp.Loss]:
    """
    1バッチ分の学習ステップを実施する関数。
    損失とその勾配をjax.value_and_gradで計算し、apply_gradientsでパラメータ更新を行う。
    """
    loss, grads = jax.value_and_grad(loss_fn)(
        state.params, batch, state.apply_fn)
    new_state = state.apply_gradients(grads=grads)
    return new_state, loss


final_state, final_loss = mlp.train_and_eval(
    dataset.load_mnist,
    use_state,
    train_step,
    loss_fn,
    mlp.OptimizableParams(learning_rate=1e-3, hidden_sizes=[128, 64]),
    5
)

print("Training completed. Final model loss:")
print(final_loss)

print("Baysian optimization start.")
final_state, final_loss = mlp.bayesian_optim(
    dataset.load_mnist, use_state, train_step, loss_fn)

2025-03-17 10:32:30.394433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742207550.415509   31762 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742207550.422004   31762 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742207550.438547   31762 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742207550.438568   31762 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742207550.438570   31762 computation_placer.cc:177] computation placer alr

Initial loss: 2.3123507


Epoch 1/5: 100%|██████████| 118/118 [00:13<00:00,  8.63it/s]


Epoch 1: Average Loss = 0.6157775521278381


Epoch 2/5: 100%|██████████| 118/118 [00:08<00:00, 13.30it/s]


...

Epoch 3: Average Loss = 0.06113035976886749


Epoch 4/5: 100%|██████████| 118/118 [00:09<00:00, 12.44it/s]


Epoch 4: Average Loss = 0.05003752186894417


Epoch 5/5: 100%|██████████| 118/118 [00:10<00:00, 11.61it/s]

Epoch 5: Average Loss = 0.046506889164447784
Final evaluation with best parameters (5 epochs):
Final loss: 0.04650689
